In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from numpy import *
from scipy import stats
import os
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, ParameterGrid
import time

# for plotting
import plotly
plotly.tools.set_credentials_file(username="majidpy", api_key="rW7nc9CHax4Z6NugEDvT")
import plotly.graph_objs as go
import plotly.plotly as py
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def stat(t):
    
    """
    This function converts the time signal to some of its statistical properties. 
    Input:
        t: time series of the acoustics signal. Size (150_000,)
    Output:
        A numpy array of size (11,)
    """
        
    mean_ = t.mean()        
    std_ = t.std()
    kurt_ = stats.kurtosis(t)
    skew_ = stats.skew(t)
    percentile_ = np.quantile(t, [0.01, 0.09, 0.25, 0.5, 0.75, 0.91, 0.99])
    t_ = np.concatenate((np.array([mean_, std_, kurt_, skew_]), 
                         percentile_))
    return t_

In [3]:
def get_quake_index(df):
    
    """
    This function checks whether or not an earthquake is happening within a 
    window in the signal. Earthquake happens when the derivitive of time signal becomes 
    positive.
    
    Input:
        df: dataframe containing acoustics signal and time to failure
    Output: 
        failed_idx: the index at which an earthquake has occured.
    """
    
    failed_index_pd = pd.DataFrame()
    failed_index_pd['time'] = df['time_to_failure']
    failed_index_pd['shifted'] = df['time_to_failure'].shift(1, axis=0)
    failed_index_pd['diff'] = failed_index_pd['time']- failed_index_pd['shifted']
    failed_index_pd = failed_index_pd.dropna()
    failed_idx = failed_index_pd.loc[failed_index_pd['diff']>0].index.min()
    return failed_idx

In [4]:
def prep_data(arg):
    
    """
    This function reads the training data peice by peice in the chuncks of (150_000,) and
    converts the time signal and its derivitive to their statistical properties. 
    Input: 
        arg: a dictionary of start and end index. Used in multi-threading
    Output: 
        X_processed: numpy array of (m, 22) where m = (end-start)/150_000
        y_processed: numpy array of (m,)
        quake_index: indeces at which an earthquake has happened. Not used in this version     
    """
        
    start = arg['start']
    end = arg['end']
    
    i = 1
    quake_index = []
    X_processed = []
    y_processed = []
    try:
        train_csv = pd.read_csv('train.csv', nrows=int(150e+3), header = 0,
                               skiprows = range(1, int(start)))
        has_data = train_csv.shape[0]
    except:
        has_data = 0
        print('end of data at {}'.format([start]))
    while has_data:

        # adding derivitive
        train_csv['shift'] = train_csv['acoustic_data'].shift(1, axis=0)
        train_csv['acoustic_drvt'] = train_csv['acoustic_data'] - train_csv['shift']
        train_csv = train_csv.dropna()
        train_csv = train_csv.drop(['shift'], axis=1)

        # checking to see if quake has happened within this range
        idx = get_quake_index(train_csv)
        if not isnan(idx):
            quake_index.append((start, i, idx))
            train_csv1 = train_csv[:idx-4]
            t1 = stat(train_csv1['acoustic_data'])
            t2 = stat(train_csv1['acoustic_drvt'])
            X_processed.append(np.concatenate((t1, t2)))
            y_processed.append(train_csv1.iloc[-1]['time_to_failure'])

            train_csv2 = train_csv[idx+4:]
            t1 = stat(train_csv2['acoustic_data'])
            t2 = stat(train_csv2['acoustic_drvt'])
            X_processed.append(np.concatenate((t1, t2)))
            y_processed.append(train_csv2.iloc[-1]['time_to_failure'])
        else:
            t1 = stat(train_csv['acoustic_data'])
            t2 = stat(train_csv['acoustic_drvt'])
            X_processed.append(np.concatenate((t1, t2)))
            y_processed.append(train_csv.iloc[-1]['time_to_failure'])

        row_2_skip = int(start+ i*150e+3)
        if row_2_skip < end:            
            i+=1
            if i%50==0:
                print(i)
            try:
                train_csv = pd.read_csv('train.csv', nrows=int(150e+3), header = 0, 
                                    skiprows = range(1, row_2_skip))
                has_data = train_csv.shape[0]
            except:
                has_data = 0
        else:
            has_data = 0
    del train_csv
    return X_processed, y_processed, quake_index

In [ ]:
from multiprocessing import Pool

"""
There are 700M data in the training set. Through this multiprocessing, we will only process
150M of them. This was the limit of my memory. If memory is not an issue, then the list in 
args can be continued to 7000*150e+3. Else, This block can be repeated and after every block
results can be saved to append at the end. 
"""

args = [{'start': 1, 'end': 100*150e+3}, 
        {'start': 100*150e+3, 'end': 200*150e+3},
        {'start': 200*150e+3, 'end': 300*150e+3},
        {'start': 300*150e+3, 'end': 400*150e+3},
        {'start': 400*150e+3, 'end': 500*150e+3},
        {'start': 500*150e+3, 'end': 600*150e+3},
        {'start': 600*150e+3, 'end': 700*150e+3},
        {'start': 700*150e+3, 'end': 800*150e+3},
        {'start': 800*150e+3, 'end': 900*150e+3},
        {'start': 900*150e+3, 'end': 1000*150e+3}]
   
p = Pool(processes=os.cpu_count())
data_chunk = p.map(prep_data, args)
y_time = []
X_processed = []
for batch in data_chunk:
    y_time.extend(batch[1])
    X_processed.extend(batch[0])
X_processed_np = np.asarray(X_processed)
y_processed_np = np.array(y_time)


In [5]:
"""
Since processing take a long time, I saved processed numpy arrays and just load them
"""
X_train_all = np.load('X_processed_np.npy')
y_train_all = np.load('y_processed_np.npy')

In [6]:
print('X_train size: {} \ny_train size: {}'.format(X_train_all.shape, y_train_all.shape))

X_train size: (2710, 22) 
y_train size: (2710,)


In [7]:
trace1 = go.Scatter(y = X_train_all[:, 0])
trace2 = go.Scatter(y = X_train_all[:, 1])
trace3 = go.Scatter(y = X_train_all[:, 4])
trace4 = go.Scatter(y = X_train_all[:, 10])
trace5 = go.Scatter(y = y_train_all)
data = [trace1, trace2, trace3, trace4, trace5]

fig = plotly.tools.make_subplots(rows=3, cols=2, 
                                 specs=[[{}, {}], [{}, {}], [{'colspan':2}, None]], 
                                subplot_titles=('Mean','Std. Deviation', '1st percentile',
                                               '99th percentile', 'time to quake'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 1)

plotly.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5           -      ]



In [8]:
# normalizing the input matrix
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_all)

In [9]:
# splitting to train and test 
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, 
                                                    y_train_all, test_size = 0.2)

In [10]:

import lightgbm
def model_lgbm(lr = 0.1, ml = 31, mdl = 20, bfr = 1.0, bfq = 0, ff = 1.0, esr = 200):
    model = lightgbm.LGBMRegressor(num_iterations=20000, random_state=42, num_threads = 4,
                                  learning_rate = lr, max_leaves = ml, min_data_in_leaf = mdl,
                                  bagging_fraction = bfr, bagging_freq = bfq, 
                                  feature_fraction = ff, early_stopping_rounds = esr)
    
    return model

In [59]:
"""
Gradient boosting method. We won't use any of the bagging parameters. 
Some of the parameters such as learning_rate and max_leaves are selected by tunning 
hyper-parameters, as layed-out below
"""
def model_lgbm_tune(kwargs):
    """
    This model takes an argument among LGBMRegressor parametrs and makes a model based
    on the input argument. 
    """
    model = lightgbm.LGBMRegressor(num_iterations=20000, random_state=42, num_threads = 4,
                                   early_stopping_rounds = 200, **kwargs)
    
    return model


"""
Hyper-parameter searching
"""
def searchgrid(params):
    """
    Function for iterating through different values of a LGBMRegressor parametr and record
    training and validation errors.
    """    
    train_error = []
    test_error = []
    tuples = []
    keys= []
    model = model_lgbm()
    for param in list(ParameterGrid(params)):
        del model #this is important else lgb model trains from last trained model in its memory
        model = model_lgbm_tune(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], 
                  eval_metric='l1', verbose=-1)
        train_error.append(model.best_score_['training']['l1'])
        test_error.append(model.best_score_['valid_1']['l1'])
        
        temp_ = []
        key_ = []
        for key, value in param.items():
            temp_.append(value)
            key_.append(key)
        tuples.append(temp_)
        keys.append(key_)
    
    return train_error, test_error, tuples, keys

In [ ]:
"""
We run this only when tunning hyper-parametrs. After tunning, 
only the optimum parameters are used.
"""
train_error, test_error, tuples, keys = searchgrid({'min_data_in_leaf' : [5, 10, 20, 50, 75, 100, 200], 
                                     'learning_rate': [0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.5], 
                                     'max_leaves': [5, 10, 30, 50, 80, 100, 500]})


In [63]:
"""
Plotting training and validation error to find optimized variables
"""
tuples_ = [str(t) for t in tuples]
trace1 = go.Scatter(x= tuples_, y = train_error, name='train_error')
trace2 = go.Scatter(x= tuples_, y = test_error, name = 'validation_error', yaxis='y2')
layout = go.Layout(title='Grid Search',
                   xaxis=dict(type = "category", 
                              title='[learning_rate, max_leaves, min_data_in_leaf]', 
                              titlefont=dict(size=18,)),
                   yaxis=dict(title='Train Error'),
                   yaxis2=dict(
                       title='Validation Error',
                       overlaying='y',
                       side='right'))
data = [trace1, trace2]
fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig)

In [64]:
"""
model_lgbm is based on optimized parameters
"""
model = model_lgbm_tune({'min_data_in_leaf' : 200, 'learning_rate': 0.05, 'max_leaves': 5})
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], 
         eval_metric='l1', verbose=100)

Training until validation scores don't improve for 200 rounds.
[100]	training's l1: 2.1447	training's l2: 7.47763	valid_1's l1: 2.29347	valid_1's l2: 8.46407
[200]	training's l1: 2.06531	training's l2: 6.98071	valid_1's l1: 2.30397	valid_1's l2: 8.58417
Early stopping, best iteration is:
[81]	training's l1: 2.171	training's l2: 7.629	valid_1's l1: 2.29207	valid_1's l2: 8.42724


/home/majid/applications/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/home/majid/applications/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument



LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       early_stopping_rounds=200, importance_type='split',
       learning_rate=0.05, max_depth=-1, max_leaves=5,
       min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=200,
       min_split_gain=0.0, n_estimators=100, n_jobs=-1,
       num_iterations=20000, num_leaves=31, num_threads=4, objective=None,
       random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [65]:
"""
Plotting feature importance
"""
features = ['mean', 'std', 'kurtosis', 'skew', 'quantile_1', 
           'quantile_9', 'quantile_25', 'quantile_50', 'quantile_75', 
            'quantile_91', 'quantile_99',
           'drvt_mean', 'drvt_std', 'drvt_kurtosis', 'drvt_skew', 'drvt_quantile_1', 
           'drvt_quantile_9', 'drvt_quantile_25', 'drvt_quantile_50', 'drvt_quantile_75', 
            'drvt_quantile_91', 'drvt_quantile_99']

t = pd.DataFrame(columns=['feature', 'importance'])
t['feature'] = features
t['importance'] = model.feature_importances_
t = t.sort_values(by='importance', ascending=True)

trace1 = go.Bar(x = t['importance'], 
                y = t['feature'], orientation = 'h')
data = [trace1]
fig = dict(data=data)

plotly.offline.iplot(fig)

In [66]:
# reading test files, processing them and sclaing them through training scaler
files = os.listdir('./test')
processed_files = {}
X_processed = []
for file in files:
    
    # reading file into pandas
    temp_ = pd.read_csv(os.path.join('./test', file))
    
    # adding derivitive
    temp_['shift'] = temp_['acoustic_data'].shift(1, axis=0)
    temp_['acoustic_drvt'] = temp_['acoustic_data'] - temp_['shift']
    temp_ = temp_.dropna()
    temp_ = temp_.drop(['shift'], axis=1)

    t1 = stat(temp_['acoustic_data'])
    t2 = stat(temp_['acoustic_drvt'])
    x_test_np = np.concatenate((t1, t2))
    
    # normalizing entries
    x_test_scaled = scaler.transform(x_test_np.reshape(1,-1))
    processed_files[file] = x_test_scaled
    
    

In [67]:
# making predictions and saving them into .csv file for submission.
submit1 = pd.read_csv('sample_submission.csv')
for key, value in processed_files.items():
    pred = model.predict(value)
    submit1.loc[submit1['seg_id']==key[:-4], 'time_to_failure'] = pred[0]

submit1.to_csv('submit11.csv', index=False)